## 00 Install

In [ ]:
# !pip install torchinfo
# !pip install modules
# !pip install pymysql sqlalchemy

## 01 Import

In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import warnings
warnings.filterwarnings("ignore")
import time
from collections import OrderedDict # ema에서 사용

import torch
import torchvision
from torchvision import models
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchinfo import summary
import torchvision.transforms as transforms

from pathlib import Path
from typing import Tuple, List, Dict, Optional
import random
from PIL import Image, ImageDraw
from torchvision.utils import make_grid

# 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 02 Load Dataset

In [ ]:
# 데이터 경로 설정
data_dir = 'your_path/fruit-and-vegetable-image'

# 데이터 불러오기
train_dir = os.path.join(data_dir, 'train')
train_filepaths = list(Path(train_dir).rglob('*.jpg')) + \
                  list(Path(train_dir).rglob('*.jpeg')) + \
                  list(Path(train_dir).rglob('*.png')) + \
                  list(Path(train_dir).rglob('*.JPG'))

test_dir = os.path.join(data_dir, 'test')
test_filepaths = list(Path(test_dir).rglob('*.jpg')) + \
                 list(Path(test_dir).rglob('*.jpeg')) + \
                 list(Path(test_dir).rglob('*.png')) + \
                 list(Path(test_dir).rglob('*.JPG'))

val_dir = os.path.join(data_dir, 'validation')
val_filepaths = list(Path(val_dir).rglob('*.jpg')) + \
                 list(Path(val_dir).rglob('*.jpeg')) + \
                 list(Path(val_dir).rglob('*.png')) + \
                 list(Path(val_dir).rglob('*.JPG'))


def proc_img(filepath):
    """
    이미지 파일의 경로와 라벨을 포함하는 DataFrame을 생성하는 함수
    """

    # 파일 경로에서 라벨(폴더명) 추출
    labels = [str(filepath[i]).split("/")[-2] for i in range(len(filepath))]

    # 파일 경로를 pandas Series로 변환하고 문자열로 저장
    filepath = pd.Series(filepath, name='Filepath').astype(str)

    # 라벨을 pandas Series로 변환
    labels = pd.Series(labels, name='Label')

    # 파일 경로와 라벨을 하나의 DataFrame으로 합치기
    df = pd.concat([filepath, labels], axis=1)

    # 데이터 프레임을 랜덤하게 섞고 인덱스 초기화
    df = df.sample(frac=1).reset_index(drop=True)

    return df

# 학습(train), 테스트(test), 검증(validation) 데이터 프레임 생성
train_df = proc_img(train_filepaths)
test_df = proc_img(test_filepaths)
val_df = proc_img(val_filepaths)

# 클래스 목록 가져오기
class_labels = train_df.Label.unique()

## 03 Create Transformer

In [ ]:
# ConvNeXt 모델의 기본 사전 학습된 가중치 불러오기
ConvNeXt_weights = models.ConvNeXt_Base_Weights.DEFAULT

# 해당 가중치에 맞는 변환(transform) 객체 가져오기
ConvNeXt_transformer = ConvNeXt_weights.transforms() # 모델이 요구하는 데이터 전처리 파이프라인이 반환

# 변환 객체 출력 (어떤 변환이 적용되는지 확인)
ConvNeXt_transformer

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
# 랜덤한 사각형 노이즈 추가 함수
def add_rectangle_noise(image):
    """이미지에 랜덤한 크기와 위치의 사각형 노이즈 추가"""
    draw = ImageDraw.Draw(image)
    width, height = image.size

    # 노이즈 크기 (전체 이미지 크기의 30~50%)
    box_width = random.randint(int(0.3 * width), int(0.5 * width))
    box_height = random.randint(int(0.3 * height), int(0.5 * height))

    # 랜덤 위치
    x1 = random.randint(0, width - box_width)
    y1 = random.randint(0, height - box_height)
    x2 = x1 + box_width
    y2 = y1 + box_height

    # 랜덤 색상 (0~255 범위)
    noise_color = tuple(np.random.randint(0, 256, size=3).tolist())

    # 사각형 그리기
    draw.rectangle([x1, y1, x2, y2], fill=noise_color)
    return image


# 데이터셋 증강 및 변환 적용
augmented_transform = transforms.Compose([
    transforms.RandomRotation(20),  # -20도 ~ 20도 회전
    transforms.RandomHorizontalFlip(),  # 50% 확률로 좌우 반전
    transforms.Lambda(lambda img: add_rectangle_noise(img)),  # 랜덤한 사각형 노이즈 추가
    ConvNeXt_transformer  # 기본 변환 적용
])

# 원본과 증강 데이터셋을 각각 생성
train_data_original = ImageFolder(root=train_dir, transform=ConvNeXt_transformer)  # 원본 데이터
train_data_augmented = ImageFolder(root=train_dir, transform=augmented_transform)  # 증강 데이터

validation_data_original = ImageFolder(root=val_dir, transform=ConvNeXt_transformer)  # 검증 원본 데이터
validation_data_augmented = ImageFolder(root=val_dir, transform=augmented_transform)  # 검증 증강 데이터

# 결합
train_data_transformed = torch.utils.data.ConcatDataset([train_data_original, train_data_augmented])
validation_data_transformed = torch.utils.data.ConcatDataset([validation_data_original, validation_data_augmented])

print(f"Original dataset size: {len(train_data_original)}")
print(f"Augmented dataset size: {len(train_data_augmented)}")
print(f"Combined dataset size: {len(train_data_transformed)}")
print(f"Validation dataset szie: {len(validation_data_transformed)}")

# 테스트 데이터는 원본 그대로 유지
test_data_transformed = ImageFolder(root=test_dir, transform=ConvNeXt_transformer)

# 변환된 데이터셋 확인
print(train_data_original, '\n\n', train_data_augmented)

Original dataset size: 3115
Augmented dataset size: 3115
Combined dataset size: 6230
Validation dataset szie: 702
Dataset ImageFolder
    Number of datapoints: 3115
    Root location: /content/drive/MyDrive/Colab_Notebooks/fruit-and-vegetable-image/train
    StandardTransform
Transform: ImageClassification(
               crop_size=[224]
               resize_size=[232]
               mean=[0.485, 0.456, 0.406]
               std=[0.229, 0.224, 0.225]
               interpolation=InterpolationMode.BILINEAR
           ) 

 Dataset ImageFolder
    Number of datapoints: 3115
    Root location: /content/drive/MyDrive/Colab_Notebooks/fruit-and-vegetable-image/train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               RandomHorizontalFlip(p=0.5)
               Lambda()
               ImageClassification(
               crop_size=[224]
               resize_size=[232]
               mean=[0.4

## 04 Dataloader

In [ ]:
# 배치 크기 및 워커(worker) 수 설정
BATCH_SIZE = 64  # 한 번에 학습할 데이터 샘플 수
NUM_WORKERS = os.cpu_count()  # 사용할 CPU 코어 개수 (최대 성능 활용)

# 학습 데이터 로더 생성
train_dataloader = DataLoader(dataset=train_data_transformed,  # 학습 데이터셋
                              batch_size=BATCH_SIZE,  # 배치 크기 설정
                              num_workers=NUM_WORKERS,  # CPU 워커 개수 설정
                              shuffle=True,  # 학습 데이터는 랜덤으로 섞어서 로드
                              pin_memory=True)  # CUDA 사용 시 메모리 핀 설정 (속도 향상)

# 검증 데이터 로더 생성
validation_dataloader = DataLoader(dataset=validation_data_transformed,
                                   batch_size=BATCH_SIZE,
                                   num_workers=NUM_WORKERS,
                                   shuffle=False,
                                   pin_memory=True)

# 테스트 데이터 로더 생성
test_dataloader = DataLoader(dataset=test_data_transformed,
                             batch_size=BATCH_SIZE,
                             num_workers=NUM_WORKERS,
                             shuffle=False,
                             pin_memory=True)

## 05 Modeling

In [ ]:
# ConvNeXt-Base 모델 생성 및 사전학습된 가중치 적용
ConvNeXt_model = models.convnext_base(weights=ConvNeXt_weights).to(device)

# 특징 추출기(Feature Extractor) 부분 동결
for param in ConvNeXt_model.features.parameters():
    param.requires_grad = False # 기존 가중치를 고정하여 학습되지 않도록 설정

# 랜덤 시드 고정
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# 분류기(Classifier) 레이어 재구성
ConvNeXt_model.classifier = nn.Sequential(
    nn.Flatten(1),  # 텐서를 1차원으로 평탄화
    nn.LayerNorm(normalized_shape=(1024,), eps=1e-06, elementwise_affine=True),  # 정규화
    nn.Linear(in_features=1024, out_features=len(class_labels)) # 출력 뉴런 수를 클래스 개수(len(class_labels))로 설정
)

# 모델 디바이스 이동
ConvNeXt_model = ConvNeXt_model.to(device)

# 모델의 구조를 다시 확인하여 변경된 부분을 확인
summary(model=ConvNeXt_model,
        input_size=[32, 3, 224, 224], # 입력 데이터 크기 (batch_size=32, 채널=3, 높이=224, 너비=224)
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],  # 출력할 정보 설정
        row_settings=['var_names']) # 변수명 출력

Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth
100%|██████████| 338M/338M [00:03<00:00, 90.7MB/s]


Layer (type (var_name))                                 Input Shape               Output Shape              Param #                   Trainable
ConvNeXt (ConvNeXt)                                     [32, 3, 224, 224]         [32, 36]                  --                        Partial
├─Sequential (features)                                 [32, 3, 224, 224]         [32, 1024, 7, 7]          --                        False
│    └─Conv2dNormActivation (0)                         [32, 3, 224, 224]         [32, 128, 56, 56]         --                        False
│    │    └─Conv2d (0)                                  [32, 3, 224, 224]         [32, 128, 56, 56]         (6,272)                   False
│    │    └─LayerNorm2d (1)                             [32, 128, 56, 56]         [32, 128, 56, 56]         (256)                     False
│    └─Sequential (1)                                   [32, 128, 56, 56]         [32, 128, 56, 56]         --                        False
│    │    └─CN

## 06 Training

In [ ]:
def update_ema(ema_model: OrderedDict, model: torch.nn.Module, decay: float):
    with torch.no_grad():
        model_state = model.state_dict()
        for name, param in model_state.items():
            if param.dtype in (torch.float16, torch.float32, torch.float64):  # EMA 대상
                if name in ema_model:
                    ema_model[name].mul_(decay).add_(param, alpha=1 - decay)
                else:
                    ema_model[name] = param.clone()

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """
    단일 학습 스텝을 수행하는 함수
    """
    model.train()  # 학습 모드 설정
    train_loss, train_acc = 0, 0  # 학습 손실 및 정확도 초기화

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # 입력 데이터와 라벨을 지정된 장치로 이동
        optimizer.zero_grad()  # 이전 단계의 그래디언트 초기화
        y_pred = model(X)  # 모델 예측 수행
        loss = loss_fn(y_pred, y)  # 손실 계산
        loss.backward()  # 역전파(Backpropagation)
        optimizer.step()  # 가중치 업데이트

        train_loss += loss.item()  # 손실 합산
        acc = (y_pred.argmax(dim=1) == y).sum().item() / len(y)  # 정확도 계산
        train_acc += acc  # 정확도 합산

    return train_loss / len(dataloader), train_acc / len(dataloader)  # 평균 손실 및 정확도 반환


def eval_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """
    단일 평가 스텝을 수행하는 함수
    """
    model.eval()  # 평가 모드 설정
    eval_loss, eval_acc = 0, 0  # 평가 손실 및 정확도 초기화

    with torch.inference_mode():  # 평가 시 그래디언트 계산 비활성화 (속도 향상)
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)  # 입력 데이터와 라벨을 지정된 장치로 이동
            y_pred = model(X)  # 모델 예측 수행
            loss = loss_fn(y_pred, y)  # 손실 계산
            eval_loss += loss.item()  # 손실 합산
            acc = (y_pred.argmax(dim=1) == y).sum().item() / len(y)  # 정확도 계산
            eval_acc += acc  # 정확도 합산

    return eval_loss / len(dataloader), eval_acc / len(dataloader)  # 평균 손실 및 정확도 반환


def train_eval(model: torch.nn.Module,
               train_dataloader: torch.utils.data.DataLoader,
               val_dataloader: torch.utils.data.DataLoader,
               epochs: int,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> List[Dict[str, float]]:
    """
    지정된 에포크(epochs) 동안 모델을 학습하고 평가하는 함수
    """
    results = []  # 학습 및 평가 결과를 저장할 리스트

    ema_params = OrderedDict()  # EMA(지수이동평균)를 위한 파라미터 저장용 딕셔너리
    ema_decay = 0.9999  # EMA 적용 시 사용될 decay 값 (가중치 평균 비율)

    # 전체 학습 시작 시간 기록
    total_start_time = time.time()

    for epoch in range(epochs):
        # 학습 단계 수행
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        # EMA(지수 이동 평균) 파라미터 업데이트 – 모델 파라미터의 평균을 저장하여 안정적인 평가에 사용
        update_ema(ema_params, model, ema_decay)
        # 평가 단계 수행
        val_loss, val_acc = eval_step(model, val_dataloader, loss_fn, device)

        # 현재 학습률 확인
        current_lr = optimizer.param_groups[0]['lr']

        # 학습 및 평가 결과 출력
        print(f"Epoch {epoch+1}/{epochs}: \n"
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} \n"
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | LR: {current_lr:.6f}")

        # 결과 저장
        results.append({
            "epoch": epoch+1,
            "Train_loss": train_loss,
            "Train_acc": train_acc,
            "Val_loss": val_loss,
            "Val_acc": val_acc,
            "lr": current_lr
        })

    # EMA 파라미터 백업 및 적용
    original_params = {name: param.data.clone() for name, param in model.named_parameters() if name in ema_params}
    for name, param in model.named_parameters():
        if name in ema_params:
            param.data.copy_(ema_params[name])

    ema_val_loss, ema_val_acc = eval_step(model, val_dataloader, loss_fn, device)
    print(f"EMA Validation Accuracy: {ema_val_acc:.4f} | EMA Validation Loss: {ema_val_loss:.4f}")

    # 원래 파라미터 복원
    for name, param in model.named_parameters():
        if name in original_params:
            param.data.copy_(original_params[name])


    # 전체 학습 끝 시간 기록
    total_end_time = time.time()
    print(f"\n🕒 전체 학습 시간: {(total_end_time - total_start_time) / 60:.2f}분")

    return results

### train()

In [ ]:
# 손실 함수 및 옵티마이저 초기화
loss_fn = nn.CrossEntropyLoss()  # 다중 분류 문제에 적합한 크로스 엔트로피 손실 함수 사용
optimizer = torch.optim.AdamW(ConvNeXt_model.parameters(), lr=0.001)  # AdamW 옵티마이저 사용, 학습률(lr) 설정

# 학습할 총 에포크 수 설정
NUM_EPOCHS = 50

# 학습 실행
results = train_eval(model=ConvNeXt_model,
                     train_dataloader=train_dataloader,
                     val_dataloader=validation_dataloader,
                     epochs=NUM_EPOCHS,
                     loss_fn=loss_fn,
                     optimizer=optimizer,
                     device=device)

Epoch 1/50: 
Train Loss: 1.1046 | Train Acc: 0.7161 
Val Loss: 0.4095 | Val Acc: 0.8731 | LR: 0.001000
Epoch 2/50: 
Train Loss: 0.4602 | Train Acc: 0.8586 
Val Loss: 0.3043 | Val Acc: 0.8963 | LR: 0.001000
Epoch 3/50: 
Train Loss: 0.3620 | Train Acc: 0.8833 
Val Loss: 0.2685 | Val Acc: 0.9062 | LR: 0.001000
Epoch 4/50: 
Train Loss: 0.3048 | Train Acc: 0.9031 
Val Loss: 0.2526 | Val Acc: 0.9104 | LR: 0.001000
Epoch 5/50: 
Train Loss: 0.2645 | Train Acc: 0.9133 
Val Loss: 0.2432 | Val Acc: 0.9119 | LR: 0.001000
Epoch 6/50: 
Train Loss: 0.2427 | Train Acc: 0.9216 
Val Loss: 0.2131 | Val Acc: 0.9204 | LR: 0.001000
Epoch 7/50: 
Train Loss: 0.2206 | Train Acc: 0.9270 
Val Loss: 0.1940 | Val Acc: 0.9375 | LR: 0.001000
Epoch 8/50: 
Train Loss: 0.1998 | Train Acc: 0.9335 
Val Loss: 0.1982 | Val Acc: 0.9148 | LR: 0.001000
Epoch 9/50: 
Train Loss: 0.1909 | Train Acc: 0.9345 
Val Loss: 0.1894 | Val Acc: 0.9303 | LR: 0.001000
Epoch 10/50: 
Train Loss: 0.1856 | Train Acc: 0.9329 
Val Loss: 0.2075 | 

## 07 Result

In [ ]:
# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

In [ ]:
# 테스트 셋 평가
test_loss, test_acc = eval_step(model=ConvNeXt_model,
                                dataloader=test_dataloader,
                                loss_fn=loss_fn,
                                device=device)

In [ ]:
# 결과
print(f"Test accuracy: {test_acc:.6f}")
print(f"Test Loss: {test_loss:.6f}")

Test accuracy: 0.966146
Test Loss: 0.147261


## 08 DB 저장

In [ ]:
from sqlalchemy import create_engine

# DB 연결 정보 입력 (Aiven MySQL 기준)
host = 'YOUR_AIVEN_HOST'
port = 'YOUR_AIVEN_PORT'
username = 'YOUR_USERNAME'
password = 'YOUR_PASSWORD'
database = 'YOUR_DATABASE'

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")
# DataFrame을 MySQL에 저장
results_df.to_sql(name='train_logs_convnextnet', con=engine, if_exists='append', index=False)

# 테스트 결과를 DataFrame으로 변환
test_result_df = pd.DataFrame([{
    "model_name": "ConvNeXt",
    "test_loss": test_loss,
    "test_acc": test_acc
}])

test_result_df.to_sql(name='test_results', con=engine, if_exists='append', index=False)

1